In [1]:
#import libraries
import os
import glob
import matplotlib.image as mpimg
from sklearn.model_selection import train_test_split
import numpy as np
from PIL import Image

In [18]:
#getting labels of images
directory=r'D:/books/YaleFaceDatabase/'
labels=os.listdir(directory)

In [19]:
#Some data preprocessing - removing the emotion associated with every subject eg subject01.sleepy to subject01
for i in range(len(labels)):
    s=labels[i]
    j=s.find('.')
    s=s[:j]
    labels[i]=s
#labels

In [20]:
#getting the filenames again
names=os.listdir(directory)

In [21]:
#getting paths for all images to load them.

for i in range(len(names)):
    s=names[i]
    s=directory+s
    names[i]=s
#names

The below function does the following steps:
1. Read the image of shape (243,320).
2. Flatten the image to vector of length 77760.
3. Construct the train_images matrix and test_images matrix of 115 and 50 images each.
4. It also returns the train and test labels.

In [6]:
#function to get the starting matrices
def get_start_matrices(names,labels):
    train,test,train_label,test_label=train_test_split(names,labels,train_size=0.7,shuffle=True)
    train_images=[]
    test_images=[]
    for im in train:
        train_images.append(np.array(Image.open(im).convert("L")))
    for im in test:
        test_images.append(np.array(Image.open(im).convert("L")))
    train_images=np.array(train_images)
    train_images=np.array([x.flatten() for x in train_images])
    test_images=np.array(test_images)
    test_images=np.array([x.flatten() for x in test_images])
    return train_images,test_images, train_label,test_label   
    #print(train[0],train_label[0])

Below function does following steps:
1. Let A be the train matrix of shape (115,77760). It computes the mean across every row (axis=1) and subtracts the mean from the matrix. We use the broadcasting feature of python to subtract the mean vector from the train matrix.
2. We compute the matrix L as $A^{T}$ $A$ instead of $A$ $A^{T}$. This method is described by the authors themselves in another paper of theirs - Turk, M., & Pentland, A. (1991). Eigenfaces for recognition. Journal of cognitive neuroscience, 3(1), 71-86. This method helps reduce the calculations by giving us a matrix of size (115,115) rather than using the other matrix of size (77760,77760). 
3. We then find the Eigen values E_val and eigen vectors E_vec of the matrix L. The eigenvectors E_vec_U of C = $A$ $A^{T}$ are given as A* E_vec as described by the authors.
4. Then we normalise the eigenvectors and sort them as per the eigenvalues. We sort as np.argsort(-E_val) as argsort will sort the values in ascending order, but we need the order descending. So we put in a negative sign so that the negative values get sorted in ascending order, so that the corresponding eigenvalues are sorted in descending order. We then arrange the corresponding eigenvectors. 
5. We get the set of 115 eigenvectors.


In [7]:
def E_faces(train_images, train_label):
    train_images = train_images.T
    train_mean = train_images.mean(axis=1, keepdims=True)
    #using python broadcasting to subtract mean
    train_images = train_images - train_mean
    L = np.dot((train_images).T, train_images)
    E_val, E_vec = np.linalg.eigh(L)
    E_vecs_U = np.matmul(train_images, E_vec)
    norm = np.sum(E_vecs_U ** 2, axis=0)
    norm = norm ** (1 / 2)
    E_vecs_U = E_vecs_U / norm
    indexes = np.argsort(-E_val)
    E_val = E_val[indexes]
    E_vecs_U = E_vecs_U[:, indexes]
    return E_val, E_vec,E_vecs_U, train_mean

Here, we select the k best eigenfaces V from the eigenface set. We also find the eigenface subspace using $V^{T}$ $A$. This is analogous to getting the coefficients when we represent each face in the training set as a linear combination of the k eigenfaces i.e. of shape (40,115).

In [8]:
def get_k_components(n_components,E_vecs_U,train_images):
    k_vecs=k_vecs=E_vecs_U[:,:n_components]
    weights = np.matmul(k_vecs.T, train_images.T- train_mean)
    return k_vecs, weights

Here, we perform the predictions for the test set.
1. Subtract the mean from the test images to get matrix T
2. We then project the test faces onto the eigenface subspace using $V^{T}$ $T$ to get a matrix test_weights_all of the weights of shape (k,50).
3. Then we pick the weights of a single test face using the matrix test_weight_single of shape (k,1). 
4. We subtract the test_weight from the weights of the training set in the matrix diff and square it. 
5. Then we add up the elements down every column and take square root to get the euclidean distance of the test face from every face. Then we pick the minimum distance using min and pick the corresponding predicted label using argmin. If the distance is below some threshold then we assign that label to that face and check the final accuracy after iterating for every face. The accuracy won't be consistent as the train set is obtained after random shuffling the dataset. So it might be possible that more instances of a particular subject enter the training set as compared to others, thereby skewing the eigenfaces. 

In [9]:
def predict(test_images, test_label, train_label, weights, k_vecs, mean):
    test_images = test_images.T- train_mean
    test_label = test_label.T
    test_weights_all = np.matmul(k_vecs.T, test_images)
    correct_predictions=0
    for i in range(0, len(test_label)):
        test_weight_single = np.resize(test_weights_all[:, i], (test_weights_all.shape[0],1))
        diff = (weights - test_weight_single) ** 2
        Euc_dist = np.sum(diff ** (1/2), axis=0)
        dist= Euc_dist.min(axis=0, keepdims=True)
        y_pred=train_label[Euc_dist.argmin(axis=0)]
        if dist < 100000:
            if test_label[i] == y_pred:
                correct_predictions+=1
    print("The accuracy obtained is %f percent" % (correct_predictions*100 / len(test_label)))

In [14]:
train_images,test_images, train_label,test_label = get_start_matrices(names,labels)

In [15]:
E_val,E_vec, E_vecs_U, train_mean=E_faces(train_images, train_label)

In [16]:
k_vecs,weights=get_k_components(40,E_vecs_U,train_images)

In [17]:
predict(test_images, np.array(test_label), np.array(train_label), weights, k_vecs, train_mean)

The accuracy obtained is 90.000000 percent
